# Step 5 - Initial EDA

After the data is acquired, it is time to perform an initial exploratory data analysis with the purpose of data cleaning. In this step, we try to find flaws and problems with the data, and in the next step we will try to rectify them.

# 5.1 - Problem: Missing Data

One of the most prevalent problems in any dataset is missing data, thus we'll check the number and the percentage of missing data by feature, and based on that we will make a decision.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# Load data
data_path = Path("../data/creditcard.csv")
df = pd.read_csv(data_path)

# Checking if the data has been loaded successfully by outputing the shape and size
print(f"Dataset shape: {df.shape}")
print(f"Dataset size: {df.size:,}")

Dataset shape: (284807, 31)
Dataset size: 8,829,017


In [3]:
# Printing the number of missing data
missing_count = df.isnull().sum()
print("\nNumber of missing data per feature:")
print(list(missing_count))


Number of missing data per feature:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 5.1 Conclusion

We can see that there are no missing data in this dataset and thus we can move on to the next point in our pre-cleaning EDA.

In case missing data was present, I would display a distribution of the percentage of missing value per feature to have a better understanding of the problem I'm dealing with and usually would consult a stakeholder and ask for an explanation. Sometimes there's a pattern to the missing values. Sometimes the problem is systematic. And I always check whether we have missing values in the target value when I'm dealing with supervised learning.

## 5.2 Problem: Data Type and Format Issues

The second most common problem I've faced are:

1. Mixed data types within columns.
2. Numerical data stored as strings.
3. Incorrectly stored data/time values.
4. Hidden characters and encoding issues.
5. Multiple boolean variations (0/1, T/f, ...).
6. Currency or percentage issues.

For our project here, we'll first check the datatypes of features. Then we'll check if they match with dataset description. If we see inconsistencies we'll move to the next steps.

So, for the ['Kaggle Credit Card Fraud'](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud) dataset, the dataset is advertised as:

> It contains only numerical input variables which are the result of a PCA transformation.
> Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'.
> Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset.
> The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning.
> Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Thus:

1. We'll check whether we have all features by printing the feature names.
2. We'll check if V1 to V28 are all numerical.
3. We'll check if time feature are numerical.
4. We'll check if the Class feature are all 1s and 0s.

In [4]:
# Printing the feature names
print("Feature names:")
print(list(df.columns))

selection_mask = ['Time', 'Amount', 'Class']
pca_features = df.drop(columns=selection_mask)

print(f"\nPCA Features (V1-V28) data types:")
print(f"Unique data types: {pca_features.dtypes.unique()}")
print(f"Number of features: {len(pca_features.columns)}")

print("\nTime Feature data type:")
print(df['Time'].dtype)

print("\nAmount Feature data type:")
print(df['Amount'].dtype)

print("\nClass Feature data type and distribution:")
print(f"Data type: {df['Class'].dtype}")
print(f"Unique values: {sorted(df['Class'].unique())}")
print(f"Value counts:\n{df['Class'].value_counts().sort_index()}")

Feature names:
['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']

PCA Features (V1-V28) data types:
Unique data types: [dtype('float64')]
Number of features: 28

Time Feature data type:
float64

Amount Feature data type:
float64

Class Feature data type and distribution:
Data type: int64
Unique values: [0, 1]
Value counts:
Class
0    284315
1       492
Name: count, dtype: int64


### 5.2 Conclusion

1. All the features mentioned in Kaggle exists as advertised.
2. All PCA features are numerical.
3. The Time feature is numerical.
4. The Amount feature is numerical.
5. The Class feature consists of only 1 and 0.

It is not a surprise that the common pre data cleaning EDA results are so far positive since we're using a curated dataset. But I always check nonetheless.

## 5.3 Obvious Data Quality Issue

The next step is to dig deeper into the quality of the data. I have a checklist of problems that lead to model underperformance or inability to converge that need to dealt with.

1. Impossible values: such as a negative age.
2. Values being outside of reasonable range.
3. Duplicate records.
4. Suspicious constant values or patterns.
5. Placeholder values.

There are more items, but these 5 could apply to our project.

Basic statistics (mean, standard deviation, min, max, etc.) are the first tool to answer most of these questions, but we need to display them in an organized way to avoid screen clutter.

In [5]:
from matplotlib import pyplot as plt

# Basic statistics for Time feature
print("1. Time Feature Statistics:")
print(df['Time'].describe())
print("="*50)

print("2. Amount Feature Statistics:")
print(df['Amount'].describe())
print("="*50)

print("3. PCA Features (V1-V28) Summary Statistics:")
pca_stats = pca_features.describe()
print(f"Count: {pca_stats.loc['count'].iloc[0]:,.0f} (same for all PCA features)")
print(f"Mean range: {pca_stats.loc['mean'].min():.4f} to {pca_stats.loc['mean'].max():.4f}")
print(f"Std range: {pca_stats.loc['std'].min():.4f} to {pca_stats.loc['std'].max():.4f}")
print(f"Min range: {pca_stats.loc['min'].min():.4f} to {pca_stats.loc['min'].max():.4f}")
print(f"Max range: {pca_stats.loc['max'].min():.4f} to {pca_stats.loc['max'].max():.4f}")
print("="*50)

# Check for duplicate records
print("4. Duplicate Records Check:")
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

1. Time Feature Statistics:
count    284807.000000
mean      94813.859575
std       47488.145955
min           0.000000
25%       54201.500000
50%       84692.000000
75%      139320.500000
max      172792.000000
Name: Time, dtype: float64
2. Amount Feature Statistics:
count    284807.000000
mean         88.349619
std         250.120109
min           0.000000
25%           5.600000
50%          22.000000
75%          77.165000
max       25691.160000
Name: Amount, dtype: float64
3. PCA Features (V1-V28) Summary Statistics:
Count: 284,807 (same for all PCA features)
Mean range: -0.0000 to 0.0000
Std range: 0.3301 to 1.9587
Min range: -113.7433 to -2.6046
Max range: 2.4549 to 120.5895
4. Duplicate Records Check:
Number of duplicate rows: 1081


### 5.3 Initial Conclusion

The basic analysis answers some of my questions.

1. There are no impossible negative values in Time or Amount.
2. There are transactions with amount of zero which need to be investigated.
3. There are 1,081 duplicated rows which need to be investigatged.
4. PCA features have a mean of zero which means the preprocessing was done properly and we do not need to re-center or re-scale the features which is good.

So points 2 and 3 need to be investigated.

2. **Zero-amount transactions:** I researched and found no issue with these transactions. Transactions can be zero when coupons are used or are some sort of a test transaction that validates debit/credit cards.
3. **Duplicated rows:** There are several logical reasons why duplicate records could exist in the record and are not a concern.
    *  After PCA transformation, different original transactions might round to identical values due to floating-point precision limits.
    *  Time is recorded in seconds and multiple transactions happening within the same second with the same amount would appear identical.
    *  Same person can buy the same item or a recurring payment or common transaction amounts.
    *  Common payment processing patterns such as batch processing by merchants, failed transactions that were immediately retried, and authroization holds followed by actual charges.

Thus we need to now investigate the duplicated rows.

In [8]:
print("4.1 Analyzing Duplicate Records:")

# Check if duplicates are in fraud vs non-fraud
duplicated_mask = df.duplicated(keep=False)
duplicate_df = df[duplicated_mask]

print(f"Fraud vs Non-fraud in duplicates:")
print(duplicate_df['Class'].value_counts())
print("="*35)

# Check time distribution of duplicates
print(f"Time range of duplicates:")
print(f"Min time: {duplicate_df['Time'].min()}")
print(f"Max time: {duplicate_df['Time'].max()}")
print("="*35)

# Check if duplicates cluster around certain amounts
print(f"Most common amounts in duplicates:")
print(duplicate_df['Amount'].value_counts().head())

4.1 Analyzing Duplicate Records:
Fraud vs Non-fraud in duplicates:
Class
0    1822
1      32
Name: count, dtype: int64
Time range of duplicates:
Min time: 26.0
Max time: 172233.0
Most common amounts in duplicates:
Amount
1.00      180
150.00     28
0.00       28
29.00      28
0.76       26
Name: count, dtype: int64


### 5.3 Final Conclusion

The results show strong evidence for legitimacy.

1. Transaction patterns are realistic. $1 appears 180 times which makes perfect sense.
2. Low fraud rate in duplicates which is normal and also could indicate fraudsters using common amount to avoid detection.
3. Duplicates are distributed across the span of dataset which is good.
4. The other common amounts in duplicates are look realistic.

Thus the matter is resolved.

If the duplicates were problematic and needed to fixed we needed to run more tests to find a solution and consult stakeholders on the matter. But I'd like to list a few of the common approaches I would take:

1. When duplicates are clearly data entry errors, complete removal can be used.
2. When the timing is important and for example the most recent transaction are more important I'd only keep the last occurance.
3. If duplicates in a subset of fields is not acceptable, we'll create a mask of that subset and drop the corresponding rows with the feature mask value being duplicates.
4. Sometimes duplicates in a class are still important and thus we'll only drop duplicates from the other class or classes.

## 5.4 Structural Issues

Sometimes the data has structural issue which usually happens when data is gathered from multiple sources and thus could lead to: Inconsistent column names or ordering, variable schema across different files, misaligned data from joins or merges, header row issues or missing headers, extra or missing columns, and so on.

None of these are present in our dataset here so we can skip checking for them.

## 5.5 Target Variable Quality

The last but not least important check before data cleaning is the target variable quality check. Usually I check for:

1. Missing target values and whether there's a pattern to them. We have established in this notebook that there is no such problem.
2. Target variable distribution and balance problem. Due to the nature of our current project there imbalance is a given and part of the problem and no rectifying steps can be taken in the data cleaning step.
3. Inconsistency of target labeling. Again, we have seen that there is no such problem in this project.

There is no problem with the target variable quality.